## 1.소개
- kaggle 에서 특정 데이터에 대해 kernels 에 올라와 있는 분석에 대해 공부
- 데이터를 보는 과정 및 코드 분석을 위함
- 원본 : [kaggle LOL DataAnalysis](https://www.kaggle.com/nstuber16/d/chuckephron/leagueoflegends/dataanalysisfun)
- 주제 : 포지션별 챔피언 승률 확인

In [1]:
import numpy as np
import pandas as pd
from subprocess import check_output

In [2]:
print(check_output(["ls", "data"]).decode('utf8'))

LeagueofLegends.csv
_Columns.csv
_LeagueofLegends.csv
banValues.csv
deathValues.csv
goldValues.csv
objValues.csv



In [3]:
rawData = pd.read_csv("data/LeagueofLegends.csv")

## 데이터 탐색

In [4]:
blueWins = rawData[rawData['bResult'] == 1] # 블루진영 이긴팀
redWins = rawData[rawData['rResult'] == 1] # 레드진영 이긴팀

In [6]:
blueWins.head()

,MatchHistory,League,Season,Year,blueTeamTag,bResult,rResult,redTeamTag,gamelength,blueTop,...,redTop,redTopChamp,redJungle,redJungleChamp,redMiddle,redMiddleChamp,redADC,redADCChamp,redSupportChamp,redSupport
0,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,TSM,1,0,C9,40,Dyrus,...,Balls,Gnar,Meteos,Elise,Hai,Fizz,Sneaky,Sivir,Thresh,LemonNation
2,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,WFX,1,0,GV,40,Flaresz,...,Hauntzer,Sion,Saintvicious,LeeSin,Keane,Azir,Cop,Corki,Janna,BunnyFuFuu
4,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,CLG,1,0,T8,35,Benny,...,CaliTrlolz8,Sion,Porpoise8,RekSai,Slooshi8,Lulu,Maplestreet8,Corki,Annie,Dodo8
6,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,CST,1,0,WFX,39,Cris,...,Flaresz,Renekton,ShorterACE,LeeSin,Pobelter,Leblanc,Altec,Tristana,Nami,Gleeb
7,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,TL,1,0,CLG,43,Quas,...,Benny,Rumble,Xmithie,Rengar,Link,Leblanc,Doublelift,Sivir,Blitzcrank,Aphromoo


In [7]:
redWins.head()

,MatchHistory,League,Season,Year,blueTeamTag,bResult,rResult,redTeamTag,gamelength,blueTop,...,redTop,redTopChamp,redJungle,redJungleChamp,redMiddle,redMiddleChamp,redADC,redADCChamp,redSupportChamp,redSupport
1,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,CST,0,1,DIG,38,Cris,...,Gamsu,Irelia,Crumbzz,JarvanIV,Shiphtur,Azir,CoreJJ,Corki,Annie,KiWiKiD
3,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,TIP,0,1,TL,41,Rhux,...,Quas,Gnar,IWDominate,Nunu,Fenix,Lulu,KEITH,KogMaw,Janna,Xpecial
5,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,DIG,0,1,TIP,24,Gamsu,...,Rhux,Gnar,Rush,JarvanIV,XiaoWeiXiao,Lulu,Apollo,Corki,Thresh,Adrian
8,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,C9,0,1,GV,41,Balls,...,Hauntzer,Kassadin,Saintvicious,Vi,Keane,Zed,Cop,Corki,Morgana,Bunny FuFuu
13,http://matchhistory.na.leagueoflegends.com/en/...,North_America,Spring_Season,2015,TIP,0,1,CLG,31,Impact,...,ZionSpartan,Sion,Xmithie,JarvanIV,Link,Cassiopeia,Doublelift,Kalista,Blitzcrank,Aphromoo


In [11]:
# 각 진영의 팀별 이긴횟수
bluewinsByTeam = blueWins[['blueTeamTag', 'bResult']].groupby('blueTeamTag', as_index=False).count()
bluewinsByTeam.sort_values('bResult', ascending=False).head()

,blueTeamTag,bResult
77,SKT,107
98,kt,77
12,CLG,71
89,TSM,70
31,H2K,67


In [12]:
redwinsByTeam = redWins[['redTeamTag', 'rResult']].groupby('redTeamTag', as_index=False).count()
redwinsByTeam.sort_values('rResult', ascending=False).head()

,redTeamTag,rResult
76,SKT,106
97,kt,69
87,TSM,65
7,C9,62
94,ahq,57


## 3. 포지션 별 승리 퍼센트 확인

### 3.1 미드 챔프

In [13]:
blueWinsByMiddleChamp = blueWins[['blueMiddleChamp', 'bResult']].groupby('blueMiddleChamp', as_index=False).count()

In [18]:
blueWinsByMiddleChamp.rename(columns={'blueMiddleChamp' : 'middleChamp'}, inplace=True)

In [19]:
blueWinsByMiddleChamp.head()

,middleChamp,bResult
0,Ahri,83
1,Anivia,16
2,AurelionSol,15
3,Azir,171
4,Camille,1


In [20]:
redWinsByMiddleChamp = redWins[['redMiddleChamp', 'rResult']].groupby('redMiddleChamp', as_index=False).count()
redWinsByMiddleChamp.rename(columns={'redMiddleChamp' : 'middleChamp'}, inplace=True)

In [22]:
winsByMiddleChamp = pd.merge(blueWinsByMiddleChamp, redWinsByMiddleChamp, how='left', on='middleChamp')
winsByMiddleChamp['totalWins'] = winsByMiddleChamp.bResult + winsByMiddleChamp.rResult
winsByMiddleChamp.sort_values('totalWins', ascending=False).head()

,middleChamp,bResult,rResult,totalWins
44,Viktor,181,157.0,338.0
3,Azir,171,133.0,304.0
22,Leblanc,114,108.0,222.0
34,Ryze,115,81.0,196.0
31,Orianna,96,87.0,183.0


In [24]:
# 플레이한 게임횟수 별 챔피언(블루팀)
blueGamesPlayedMiddleChamp = rawData[['blueMiddleChamp', 'MatchHistory']].groupby('blueMiddleChamp', as_index=False).count()
blueGamesPlayedMiddleChamp.rename(columns = {'MatchHistory' : 'blueGamesPlayed', 'blueMiddleChamp' : 'middleChamp'}, inplace=True)
# 플레이한 게임횟수 별 챔피언(레드팀)
redGamesPlayedMiddleChamp = rawData[['redMiddleChamp', 'MatchHistory']].groupby('redMiddleChamp', as_index=False).count()
redGamesPlayedMiddleChamp.rename(columns={'MatchHistory' : 'redGamesPlayed', 'redMiddleChamp': 'middleChamp'}, inplace=True)

# 데이터 합치기
totalGamesPlayedMiddleChamp = pd.merge(blueGamesPlayedMiddleChamp, redGamesPlayedMiddleChamp, how='left', on='middleChamp')
totalGamesPlayedMiddleChamp['totalGamesPlayed'] = totalGamesPlayedMiddleChamp.blueGamesPlayed+totalGamesPlayedMiddleChamp.redGamesPlayed
totalGamesPlayedMiddleChamp.sort_values('totalGamesPlayed', ascending=False).head()

,middleChamp,blueGamesPlayed,redGamesPlayed,totalGamesPlayed
48,Viktor,378,386,764
3,Azir,306,267,573
22,Leblanc,215,219,434
38,Ryze,197,178,375
33,Orianna,182,184,366


In [26]:
# 챔피언 승리횟수 합치기 
winningPercentByMiddleChamp = pd.merge(winsByMiddleChamp, totalGamesPlayedMiddleChamp, how='left', on='middleChamp')
winningPercentByMiddleChamp.drop(['bResult', 'rResult', 'blueGamesPlayed', 'redGamesPlayed'], axis=1, inplace=True)
winningPercentByMiddleChamp = winningPercentByMiddleChamp.fillna(0)

# 승률 계산
winningPercentByMiddleChamp['champWinPercentage'] = (winningPercentByMiddleChamp.totalWins / winningPercentByMiddleChamp.totalGamesPlayed)
winningPercentByMiddleChamp = winningPercentByMiddleChamp.fillna(0)
winningPercentByMiddleChamp.sort_values('champWinPercentage', ascending=False)

,middleChamp,totalWins,totalGamesPlayed,champWinPercentage
20,Kennen,3.0,3,1.000000
32,Quinn,13.0,19,0.684211
13,Irelia,8.0,12,0.666667
4,Camille,2.0,3,0.666667
26,Malzahar,73.0,112,0.651786
21,KogMaw,22.0,34,0.647059
14,JarvanIV,7.0,11,0.636364
48,Zed,130.0,205,0.634146
2,AurelionSol,29.0,46,0.630435
35,Swain,27.0,46,0.586957


### 3.2 탑 챔프

In [27]:
## 불루진영 승리 횟수 계산
blueWinsByTopChamp = blueWins[['blueTopChamp', 'bResult']].groupby('blueTopChamp', as_index = False).count()
blueWinsByTopChamp.rename(columns = {'blueTopChamp': 'topChamp'}, inplace = True)

## 레드진영 승리 횟수 계산
redWinsByTopChamp = redWins[['redTopChamp', 'rResult']].groupby('redTopChamp', as_index = False).count()
redWinsByTopChamp.rename(columns = {'redTopChamp': 'topChamp'}, inplace = True)

## 블루/레드 진영 합치기
winsByTopChamp = pd.merge(blueWinsByTopChamp, redWinsByTopChamp, how = 'left', on = 'topChamp')
winsByTopChamp['totalWins'] = winsByTopChamp.bResult + winsByTopChamp.rResult
winsByTopChamp.sort_values('totalWins', ascending = False).head()

,topChamp,bResult,rResult,totalWins
29,Maokai,300,270.0,570.0
11,Gnar,166,165.0,331.0
43,Rumble,185,126.0,311.0
45,Shen,144,119.0,263.0
31,Nautilus,117,107.0,224.0


In [28]:
## 블루 진영의 챔프별 전체게임수
blueGamesPlayedTopChamp = rawData[['blueTopChamp', 'MatchHistory']].groupby('blueTopChamp', as_index = False).count()
blueGamesPlayedTopChamp.rename(columns = {'MatchHistory': 'blueGamesPlayed', 'blueTopChamp': 'topChamp'}, inplace = True)

## 레드 진영의 챔프별 전체게임수
redGamesPlayedTopChamp = rawData[['redTopChamp', 'MatchHistory']].groupby('redTopChamp', as_index = False).count()
redGamesPlayedTopChamp.rename(columns = {'MatchHistory': 'redGamesPlayed', 'redTopChamp': 'topChamp'}, inplace = True)

## 블루/레드 진영 합치기
totalGamesPlayedTopChamp = pd.merge(blueGamesPlayedTopChamp, redGamesPlayedTopChamp, how = 'left', on = 'topChamp')
totalGamesPlayedTopChamp['totalGamesPlayed'] = totalGamesPlayedTopChamp.blueGamesPlayed + totalGamesPlayedTopChamp.redGamesPlayed
totalGamesPlayedTopChamp.sort_values('totalGamesPlayed', ascending = False).head()

,topChamp,blueGamesPlayed,redGamesPlayed,totalGamesPlayed
34,Maokai,514,553.0,1067.0
14,Gnar,314,365.0,679.0
50,Rumble,312,279.0,591.0
52,Shen,291,241.0,532.0
43,Poppy,217,229.0,446.0


In [58]:
## 챔피언 승리/전체 게임수 합치기
winningPercentageByTopChamp = pd.merge(winsByTopChamp, totalGamesPlayedTopChamp, how = 'left', on = 'topChamp')
winningPercentageByTopChamp.drop(['bResult', 'rResult', 'blueGamesPlayed', 'redGamesPlayed'], axis = 1, inplace = True)
winningPercentageByTopChamp = winningPercentageByTopChamp.fillna(0)

## 승률 계산
winningPercentageByTopChamp['champWinPercentage'] = (winningPercentageByTopChamp.totalWins / winningPercentageByTopChamp.totalGamesPlayed)
winningPercentageByTopChamp = winningPercentageByTopChamp.fillna(0) ## to deal w/ division by 0
winningPercentageByTopChamp.sort_values('champWinPercentage', ascending = False)

,topChamp,totalWins,totalGamesPlayed,champWinPercentage
51,Taliyah,2.0,2.0,1.000000
36,Pantheon,4.0,4.0,1.000000
4,Chogath,5.0,6.0,0.833333
28,Malzahar,4.0,5.0,0.800000
56,Zed,3.0,4.0,0.750000
1,Azir,4.0,6.0,0.666667
39,Rammus,20.0,31.0,0.645161
30,Morgana,17.0,27.0,0.629630
54,Yasuo,10.0,16.0,0.625000
2,Camille,44.0,73.0,0.602740


### 3.3 원딜 챔프

In [30]:
## 블루진영 승리횟수
blueWinsByADCChamp = blueWins[['blueADCChamp', 'bResult']].groupby('blueADCChamp', as_index = False).count()
blueWinsByADCChamp.rename(columns = {'blueADCChamp': 'adcChamp'}, inplace = True)

## 레드진영 승리횟수
redWinsByADCChamp = redWins[['redADCChamp', 'rResult']].groupby('redADCChamp', as_index = False).count()
redWinsByADCChamp.rename(columns = {'redADCChamp': 'adcChamp'}, inplace = True)

## 합치기
winsByADCChamp = pd.merge(blueWinsByADCChamp, redWinsByADCChamp, how = 'left', on = 'adcChamp')
winsByADCChamp['totalWins'] = winsByADCChamp.bResult + winsByADCChamp.rResult
winsByADCChamp.sort_values('totalWins', ascending = False).head()

,adcChamp,bResult,rResult,totalWins
14,Sivir,309,317.0,626.0
11,Lucian,238,207.0,445.0
0,Ashe,213,183.0,396.0
6,Jhin,205,171.0,376.0
4,Ezreal,181,149.0,330.0


In [31]:
## 블루진영 총 게임횟수
blueGamesPlayedADCChamp = rawData[['blueADCChamp', 'MatchHistory']].groupby('blueADCChamp', as_index = False).count()
blueGamesPlayedADCChamp.rename(columns = {'MatchHistory': 'blueGamesPlayed', 'blueADCChamp': 'adcChamp'}, inplace = True)

## 레드진영 총 게임횟수
redGamesPlayedADCChamp = rawData[['redADCChamp', 'MatchHistory']].groupby('redADCChamp', as_index = False).count()
redGamesPlayedADCChamp.rename(columns = {'MatchHistory': 'redGamesPlayed', 'redADCChamp': 'adcChamp'}, inplace = True)

## 합치기
totalGamesPlayedADCChamp = pd.merge(blueGamesPlayedADCChamp, redGamesPlayedADCChamp, how = 'left', on = 'adcChamp')
totalGamesPlayedADCChamp['totalGamesPlayed'] = totalGamesPlayedADCChamp.blueGamesPlayed + totalGamesPlayedADCChamp.redGamesPlayed
totalGamesPlayedADCChamp.sort_values('totalGamesPlayed', ascending = False).head()

,adcChamp,blueGamesPlayed,redGamesPlayed,totalGamesPlayed
14,Sivir,556,615.0,1171.0
11,Lucian,468,472.0,940.0
0,Ashe,366,395.0,761.0
6,Jhin,364,362.0,726.0
4,Ezreal,343,366.0,709.0


In [32]:
## 챔프별 승리횟수 및 전체게임수 합침
winningPercentageByADCChamp = pd.merge(winsByADCChamp, totalGamesPlayedADCChamp, how = 'left', on = 'adcChamp')
winningPercentageByADCChamp.drop(['bResult', 'rResult', 'blueGamesPlayed', 'redGamesPlayed'], axis = 1, inplace = True)
winningPercentageByADCChamp = winningPercentageByADCChamp.fillna(0)

## 승률계산
winningPercentageByADCChamp['champWinPercentage'] = (winningPercentageByADCChamp.totalWins / winningPercentageByADCChamp.totalGamesPlayed)
winningPercentageByADCChamp = winningPercentageByADCChamp.fillna(0) ## to deal w/ division by 0
winningPercentageByADCChamp.sort_values('champWinPercentage', ascending = False)

,adcChamp,totalWins,totalGamesPlayed,champWinPercentage
9,Kennen,8.0,12.0,0.666667
17,Urgot,36.0,61.0,0.590164
8,Kalista,323.0,582.0,0.554983
14,Sivir,626.0,1171.0,0.534586
0,Ashe,396.0,761.0,0.520368
6,Jhin,376.0,726.0,0.517906
20,Ziggs,20.0,39.0,0.512821
5,Graves,72.0,141.0,0.510638
16,Twitch,35.0,69.0,0.507246
3,Draven,15.0,30.0,0.500000


### 3.4 서폿 챔프 

In [33]:
## 블루진영 승리횟수
blueWinsBySupportChamp = blueWins[['blueSupportChamp', 'bResult']].groupby('blueSupportChamp', as_index = False).count()
blueWinsBySupportChamp.rename(columns = {'blueSupportChamp': 'supportChamp'}, inplace = True)

## 레드진영 승리횟수
redWinsBySupportChamp = redWins[['redSupportChamp', 'rResult']].groupby('redSupportChamp', as_index = False).count()
redWinsBySupportChamp.rename(columns = {'redSupportChamp': 'supportChamp'}, inplace = True)

## 합치기
winsBySupportChamp = pd.merge(blueWinsBySupportChamp, redWinsBySupportChamp, how = 'left', on = 'supportChamp')
winsBySupportChamp['totalWins'] = winsBySupportChamp.bResult + winsBySupportChamp.rResult
winsBySupportChamp.sort_values('totalWins', ascending = False).head()

,supportChamp,bResult,rResult,totalWins
0,Alistar,254,225.0,479.0
29,Thresh,230,214.0,444.0
5,Braum,226,198.0,424.0
10,Karma,162,157.0,319.0
9,Janna,128,124.0,252.0


In [34]:
## 블루진영 총게임수
blueGamesPlayedSupportChamp = rawData[['blueSupportChamp', 'MatchHistory']].groupby('blueSupportChamp', as_index = False).count()
blueGamesPlayedSupportChamp.rename(columns = {'MatchHistory': 'blueGamesPlayed', 'blueSupportChamp': 'supportChamp'}, inplace = True)

## 레드진영 총게임수
redGamesPlayedSupportChamp = rawData[['redSupportChamp', 'MatchHistory']].groupby('redSupportChamp', as_index = False).count()
redGamesPlayedSupportChamp.rename(columns = {'MatchHistory': 'redGamesPlayed', 'redSupportChamp': 'supportChamp'}, inplace = True)

## 합치기
totalGamesPlayedSupportChamp = pd.merge(blueGamesPlayedSupportChamp, redGamesPlayedSupportChamp, how = 'left', on = 'supportChamp')
totalGamesPlayedSupportChamp['totalGamesPlayed'] = totalGamesPlayedSupportChamp.blueGamesPlayed + totalGamesPlayedSupportChamp.redGamesPlayed
totalGamesPlayedSupportChamp.sort_values('totalGamesPlayed', ascending = False).head()

,supportChamp,blueGamesPlayed,redGamesPlayed,totalGamesPlayed
0,Alistar,467,507.0,974.0
5,Braum,472,458.0,930.0
34,Thresh,414,432.0,846.0
12,Karma,320,362.0,682.0
11,Janna,226,256.0,482.0


In [35]:
## 승리횟수, 게임수 합침
winningPercentageBySupportChamp = pd.merge(winsBySupportChamp, totalGamesPlayedSupportChamp, how = 'left', on = 'supportChamp')
winningPercentageBySupportChamp.drop(['bResult', 'rResult', 'blueGamesPlayed', 'redGamesPlayed'], axis = 1, inplace = True)
winningPercentageBySupportChamp = winningPercentageBySupportChamp.fillna(0)

## 승률 계산
winningPercentageBySupportChamp['champWinPercentage'] = (winningPercentageBySupportChamp.totalWins / winningPercentageBySupportChamp.totalGamesPlayed)
winningPercentageBySupportChamp = winningPercentageBySupportChamp.fillna(0) ## to deal w/ division by 0
winningPercentageBySupportChamp.sort_values('champWinPercentage', ascending = False)

,supportChamp,totalWins,totalGamesPlayed,champWinPercentage
23,Shen,23.0,35.0,0.657143
4,Brand,7.0,11.0,0.636364
11,Kennen,23.0,38.0,0.605263
28,Taric,64.0,109.0,0.587156
17,Malzahar,114.0,204.0,0.558824
26,Soraka,48.0,86.0,0.558140
6,Camille,5.0,9.0,0.555556
27,TahmKench,140.0,253.0,0.553360
19,Morgana,154.0,284.0,0.542254
18,MissFortune,41.0,78.0,0.525641


### 정글 챔프

In [36]:
## 블루진영 승리횟수
blueWinsByJungleChamp = blueWins[['blueJungleChamp', 'bResult']].groupby('blueJungleChamp', as_index = False).count()
blueWinsByJungleChamp.rename(columns = {'blueJungleChamp': 'jungleChamp'}, inplace = True)

## 레드진영 승리횟수
redWinsByJungleChamp = redWins[['redJungleChamp', 'rResult']].groupby('redJungleChamp', as_index = False).count()
redWinsByJungleChamp.rename(columns = {'redJungleChamp': 'jungleChamp'}, inplace = True)

## 합치기
winsByJungleChamp = pd.merge(blueWinsByJungleChamp, redWinsByJungleChamp, how = 'left', on = 'jungleChamp')
winsByJungleChamp['totalWins'] = winsByJungleChamp.bResult + winsByJungleChamp.rResult
winsByJungleChamp.sort_values('totalWins', ascending = False).head()

,jungleChamp,bResult,rResult,totalWins
24,RekSai,359,293.0,652.0
7,Gragas,270,249.0,519.0
4,Elise,247,250.0,497.0
14,LeeSin,176,143.0,319.0
17,Nidalee,177,108.0,285.0


In [37]:
## 블루진영 총게임수
blueGamesPlayedJungleChamp = rawData[['blueJungleChamp', 'MatchHistory']].groupby('blueJungleChamp', as_index = False).count()
blueGamesPlayedJungleChamp.rename(columns = {'MatchHistory': 'blueGamesPlayed', 'blueJungleChamp': 'jungleChamp'}, inplace = True)

## 레드진영 총게임수
redGamesPlayedJungleChamp = rawData[['redJungleChamp', 'MatchHistory']].groupby('redJungleChamp', as_index = False).count()
redGamesPlayedJungleChamp.rename(columns = {'MatchHistory': 'redGamesPlayed', 'redJungleChamp': 'jungleChamp'}, inplace = True)

## 합치기
totalGamesPlayedJungleChamp = pd.merge(blueGamesPlayedJungleChamp, redGamesPlayedJungleChamp, how = 'left', on = 'jungleChamp')
totalGamesPlayedJungleChamp['totalGamesPlayed'] = totalGamesPlayedJungleChamp.blueGamesPlayed + totalGamesPlayedJungleChamp.redGamesPlayed
totalGamesPlayedJungleChamp.sort_values('totalGamesPlayed', ascending = False).head()

,jungleChamp,blueGamesPlayed,redGamesPlayed,totalGamesPlayed
26,RekSai,672,666.0,1338.0
5,Elise,472,527.0,999.0
9,Gragas,474,496.0,970.0
16,LeeSin,319,347.0,666.0
10,Graves,303,284.0,587.0


In [38]:
## 승리횟수, 총게임수 합치기 
winningPercentageByJungleChamp = pd.merge(winsByJungleChamp, totalGamesPlayedJungleChamp, how = 'left', on = 'jungleChamp')
winningPercentageByJungleChamp.drop(['bResult', 'rResult', 'blueGamesPlayed', 'redGamesPlayed'], axis = 1, inplace = True)
winningPercentageByJungleChamp = winningPercentageByJungleChamp.fillna(0)

## 승률
winningPercentageByJungleChamp['champWinPercentage'] = (winningPercentageByJungleChamp.totalWins / winningPercentageByJungleChamp.totalGamesPlayed)
winningPercentageByJungleChamp = winningPercentageByJungleChamp.fillna(0) ## to deal w/ division by 0
winningPercentageByJungleChamp.sort_values('champWinPercentage', ascending = False)

,jungleChamp,totalWins,totalGamesPlayed,champWinPercentage
34,Warwick,5.0,6.0,0.833333
36,Zac,18.0,28.0,0.642857
30,Skarner,6.0,10.0,0.600000
33,Vi,37.0,63.0,0.587302
29,Shyvana,4.0,7.0,0.571429
17,Nidalee,285.0,516.0,0.552326
10,Ivern,29.0,53.0,0.547170
25,Rengar,175.0,326.0,0.536810
7,Gragas,519.0,970.0,0.535052
12,Khazix,199.0,384.0,0.518229


### 4. 챔피언 포지션별 승률 summary

In [40]:
## 모든 챔피언 list
middleChampions = winningPercentByMiddleChamp[['middleChamp']].rename(columns = {'middleChamp': 'champ'})
topChampions = winningPercentageByTopChamp[['topChamp']].rename(columns = {'topChamp': 'champ'})
adcChampions = winningPercentageByADCChamp[['adcChamp']].rename(columns = {'adcChamp': 'champ'})
supportChampions = winningPercentageBySupportChamp[['supportChamp']].rename(columns = {'supportChamp': 'champ'})
jungleChampions = winningPercentageByJungleChamp[['jungleChamp']].rename(columns = {'jungleChamp': 'champ'})

## 중복된 이름 제외한 나머지 챔피언에 대해 합치기
frames = [middleChampions, topChampions, adcChampions, supportChampions, jungleChampions]
allChampions = pd.concat(frames)
allChampions = allChampions.drop_duplicates().sort_values('champ').reset_index(drop = True)

In [41]:
allChampions.head()

,champ
0,Ahri
1,Alistar
2,Amumu
3,Anivia
4,Annie


In [43]:
## 포지션별 챔피언 승률 전체 데이터 합치기
allChampionsPositionWinPercentage = allChampions
allChampionsPositionWinPercentage = pd.merge(allChampionsPositionWinPercentage, winningPercentByMiddleChamp[['middleChamp', 'champWinPercentage']].rename(columns = {'middleChamp': 'champ', 'champWinPercentage': 'middleWinPercentage'}), how = 'left', on = 'champ')
allChampionsPositionWinPercentage = pd.merge(allChampionsPositionWinPercentage, winningPercentageByTopChamp[['topChamp', 'champWinPercentage']].rename(columns = {'topChamp': 'champ', 'champWinPercentage': 'topWinPercentage'}), how = 'left', on = 'champ')
allChampionsPositionWinPercentage = pd.merge(allChampionsPositionWinPercentage, winningPercentageByADCChamp[['adcChamp', 'champWinPercentage']].rename(columns = {'adcChamp': 'champ', 'champWinPercentage': 'adcWinPercentage'}), how = 'left', on = 'champ')
allChampionsPositionWinPercentage = pd.merge(allChampionsPositionWinPercentage, winningPercentageBySupportChamp[['supportChamp', 'champWinPercentage']].rename(columns = {'supportChamp': 'champ', 'champWinPercentage': 'supportWinPercentage'}), how = 'left', on = 'champ')
allChampionsPositionWinPercentage = pd.merge(allChampionsPositionWinPercentage, winningPercentageByJungleChamp[['jungleChamp', 'champWinPercentage']].rename(columns = {'jungleChamp': 'champ', 'champWinPercentage': 'jungleWinPercentage'}), how = 'left', on = 'champ')

In [44]:
allChampionsPositionWinPercentage = allChampionsPositionWinPercentage.fillna(0)

In [45]:
allChampionsPositionWinPercentage

,champ,middleWinPercentage,topWinPercentage,adcWinPercentage,supportWinPercentage,jungleWinPercentage
0,Ahri,0.490566,0.000000,0.000000,0.000000,0.000000
1,Alistar,0.000000,0.000000,0.000000,0.491786,0.000000
2,Amumu,0.000000,0.000000,0.000000,0.000000,0.000000
3,Anivia,0.500000,0.000000,0.000000,0.000000,0.000000
4,Annie,0.000000,0.000000,0.000000,0.523207,0.000000
5,Ashe,0.000000,0.000000,0.520368,0.000000,0.000000
6,AurelionSol,0.630435,0.000000,0.000000,0.000000,0.000000
7,Azir,0.530541,0.666667,0.000000,0.000000,0.000000
8,Bard,0.000000,0.000000,0.000000,0.504110,0.000000
9,Blitzcrank,0.000000,0.000000,0.000000,0.450000,0.000000
